In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re
from dotenv import load_dotenv

load_dotenv("C:/Users/Jochen/THWSkraken/credentials.env")

url = 'https://elearning.fhws.de/login/index.php'
values = {'username': os.environ.get('STUDENT_USER'),
          'password': os.environ.get('STUDENT_PASSWORD'),
          'anchor': ''}


In [8]:
sess = requests.Session()

In [10]:
sess.cookies

<RequestsCookieJar[Cookie(version=0, name='MoodleSession', value='u7qsble454bsn77k9fn11lii20', port=None, port_specified=False, domain='elearning.fhws.de', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [11]:
def getCRSFToken():
    r = sess.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup.find('input', {'name': 'logintoken'})['value']

def login():
    values['logintoken'] = getCRSFToken()
    r = sess.post(url, data=values)
    print("login:", r.status_code)
    return r

def getCourses():
    index = 0
    ajax1 = 'https://elearning.fhws.de/theme/remui/request_handler.php?action=get_courses_ajax&wdmdata={%22category%22:%22all%22,%22sort%22:null,%22search%22:%22%22,%22tab%22:true,%22page%22:{%22courses%22:0,%22mycourses%22:%22'
    #"https://elearning.fhws.de/theme/remui/request_handler.php?action=get_courses_ajax&wdmdata={'category':'all','sort':null,'search':'','tab':true,'page':{'courses':0,'mycourses':"
    ajax2 = '%22},%22pagination%22:true,%22view%22:%22grid%22,%22isFilterModified%22:true}'
    courses = []
    r = sess.get(ajax1 + str(index) + ajax2)
    print("get courses:", r.status_code)
    content = r.json()
    print(content.keys())
    print(content["pagination"])
    print(len(content["pagination"]))
    maxIndex = max(map(int, re.findall("page=(\d)", content["pagination"])))
    print("maxIndex:", maxIndex)
    print("len courses:", len(content["courses"]))
    courses.extend(content["courses"])
    index += 1
    while index <= maxIndex:
        r = sess.get(ajax1 + str(index) + ajax2)
        content = r.json()
        courses.extend(content["courses"])
        index += 1
    return courses

In [12]:
courses = getCourses()

get courses: 200
dict_keys(['pagination', 'courses', 'view'])



ValueError: max() arg is an empty sequence

In [ ]:
len(courses)

43

In [5]:
import pandas as pd
names = [re.search(">(.*)<", element["coursename"]).group(1) for element in courses].sort()
pd.DataFrame(names)

NameError: name 'courses' is not defined

In [36]:
r = sess.get('https://elearning.fhws.de/theme/remui/request_handler.php?action=get_courses_ajax&wdmdata={%22category%22:%22all%22,%22sort%22:null,%22search%22:%22%22,%22tab%22:true,%22page%22:{%22courses%22:0,%22mycourses%22:%221%22},%22pagination%22:true,%22view%22:%22grid%22,%22isFilterModified%22:true}&sesskey=6RPR7P5PT7')
print(r.json().keys())

dict_keys(['pagination', 'courses', 'view'])


In [8]:
page = sess.get(url)
print(page.cookies)
signin = BeautifulSoup(page.content, 'html.parser')
token = signin.find('input', {'name': 'logintoken'})['value']
print(token)
values['logintoken'] = token
r = requests.post(url, data=values, headers=dict(Referer=url), cookies=page.cookies)
print(r.status_code)
print(r.cookies)
print(r.history[0].cookies)
cookies = r.history[0].cookies
r = requests.get(
    'https://elearning.fhws.de/theme/remui/request_handler.php?action=get_courses_ajax&wdmdata={"category":"all","sort":null,"search":"","tab":true,"page":{"courses":0,"mycourses":0},"pagination":true,"view":null,"isFilterModified":true}',
    headers=dict(Referer='https://elearning.fhws.de/course/index.php?mycourses=1'), cookies=cookies)
print(r.status_code)
content = r.json()
print(content.keys())
print(len(content['courses']))
print(content['courses'][0])


<RequestsCookieJar[]>


TypeError: 'NoneType' object is not subscriptable

In [4]:
content["pagination"]

'<nav aria-label="Seite">\n        <ul class="pagination pagination-no-border">\n                <li class="page-item active disableClick">\n                    <a href="" class="page-link mx-sm-5">\n                        1\n                            <span class="sr-only">(aktuell)</span>\n                    </a>\n                </li>\n                <li class="page-item ">\n                    <a href="javascript://void(0);?page=1" class="page-link mx-sm-5">\n                        2\n                    </a>\n                </li>\n                <li class="page-item ">\n                    <a href="javascript://void(0);?page=2" class="page-link mx-sm-5">\n                        3\n                    </a>\n                </li>\n                <li class="page-item ">\n                    <a href="javascript://void(0);?page=3" class="page-link mx-sm-5">\n                        4\n                    </a>\n                </li>\n                <li class="page-item">\n    

In [5]:
from kraken import SoupChef
sou = SoupChef()

In [8]:
soup = sou.get_soup_from_text(content["pagination"])

In [13]:
results = soup.find_all("a")
results[0]

<a class="page-link mx-sm-5" href="">
                        1
                            <span class="sr-only">(aktuell)</span>
</a>